In [3]:
import pandas as pd
import glob, re

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Définition des lettres pour les gabarits

In [4]:
meem="م"
marbotat="ة"
waw="و"
yeh="ي"
alef="ا"
fatha="َ"
shadda="ّ"
damma="ُ"
dammatan="ٌ"
kasra="ِ"
sukun="ْ"

### Choix de la source
- Toutes les sources ont le champ contenant l'écriture dans la première colonne

In [19]:
source="SketchEngine"
# source="Wiktionary"
# source="Quran"
pdAdj=pd.read_csv("ADJ-%s.tsv"%source,encoding="utf8",sep="\t").drop_duplicates()
len(pdAdj)

4795

In [20]:
pdAdj.columns=range(len(pdAdj.columns))
adjectifs=pdAdj[0].drop_duplicates().tolist()
# print(adjectifs)

In [21]:
len(adjectifs)

4795

# Définition des gabarits
- les mots tirés de SketchEngine ne sont pas orthographiés comme ceux des deux autres CSV, il faut utiliser des expressions régulières différentes
- pour la séquence َّ , Wiktionary et Quran ont deux stratégies différentes :
    - fatha+shadda
    - shadda+fatha  
- il faut en tenir compte dans les regexp, ici en acceptant les deux ordres

In [22]:
shaddafatha="(?:"+shadda+fatha+"|"+fatha+shadda+")"
fa33aalFull="^(.)"+fatha+"(.)"+shaddafatha+alef+"(.)"+marbotat+"?"+dammatan+"?$"
fa33aalSketch=u"^(.)(.)"+alef+"(.)"+marbotat+"?$"
mif3aalFull="^"+meem+kasra+"(.)"+sukun+"(.)"+fatha+alef+"(.)"+marbotat+"?"+dammatan+"?$"
mif3aalSketch=meem+u"^(.)(.)"+alef+"(.)"+marbotat+"?$"
fa3uulFull="^(.)"+damma+"(.)"+damma+waw+"(.)"+marbotat+"?"+dammatan+"?$"
fa3uulSketch="^(.)(.)"+waw+"(.)"+marbotat+"?$"
fa3iilFull="^(.)"+fatha+"(.)"+kasra+yeh+"(.)"+marbotat+"?"+dammatan+"?$"
fa3iilSketch="^(.)(.)"+yeh+"(.)"+marbotat+"?$"
fa3ilFull="^(.)"+fatha+"(.)"+kasra+"(.)"+marbotat+"?"+dammatan+"?$"
fa3ilSketch="^(.)(.)"+yeh+"(.)"+marbotat+"?$"

if source=="SketchEngine":
    fa33aalRegex=fa33aalSketch
    mif3aalRegex=mif3aalSketch
    fa3uulRegex=fa3uulSketch
    fa3iilRegex=fa3iilSketch
    fa3ilRegex=fa3ilSketch
else:
    fa33aalRegex=fa33aalFull
    mif3aalRegex=mif3aalFull
    fa3uulRegex=fa3uulFull
    fa3iilRegex=fa3iilFull
    fa3ilRegex=fa3ilFull


In [23]:
dfIntensifs=pd.DataFrame(columns="forme gabarit c3 c2 c1".split(" "))

def extractDict(match):
    return {"forme":match.group(0),
            "c1":match.group(1),
            "c2":match.group(2),
            "c3":match.group(3),
           }
for i,form in enumerate(adjectifs):
    # print (form)
    m=re.match(fa33aalRegex,form)
    if m:
        # print (i,m.group(0),"\tfa33aal,racine :",m.group(1),m.group(2),m.group(3))
        formeDict=extractDict(m)
        formeDict["gabarit"]=["fa33aal"]
        pdFormeDict=pd.DataFrame([formeDict])
        dfIntensifs=pd.concat([dfIntensifs,pdFormeDict])
    m=re.match(mif3aalRegex,form)
    if m:
        # # print (i,m.group(0),"\t  mif3al, racine :",m.group(1),m.group(2),m.group(3))
        formeDict=extractDict(m)
        formeDict["gabarit"]=["mif3al"]
        pdFormeDict=pd.DataFrame([formeDict])
        dfIntensifs=pd.concat([dfIntensifs,pdFormeDict])
    m=re.match(fa3uulRegex,form)
    if m:
        # print (i,m.group(0),"\t    fa3uul, racine :",m.group(1),m.group(2),m.group(3))    
        formeDict=extractDict(m)
        formeDict["gabarit"]=["fa3uul"]
        pdFormeDict=pd.DataFrame([formeDict])
        dfIntensifs=pd.concat([dfIntensifs,pdFormeDict])
    if source=="SketchEngine":
        m=re.match(fa3iilSketch,form)
        if m:
            # print (i,m.group(0),"\t      fa3iil, racine :",m.group(1),m.group(2),m.group(3))            
            formeDict=extractDict(m)
            formeDict["gabarit"]=["fa3iil", "fa3il"]
            pdFormeDict=pd.DataFrame([formeDict])
            dfIntensifs=pd.concat([dfIntensifs,pdFormeDict])        
    else:
        m=re.match(fa3iilRegex,form)
        if m:
            # print (i,m.group(0),"\t      fa3iil, racine :",m.group(1),m.group(2),m.group(3))            
            formeDict=extractDict(m)
            formeDict["gabarit"]=["fa3iil"]
            pdFormeDict=pd.DataFrame([formeDict])
            dfIntensifs=pd.concat([dfIntensifs,pdFormeDict])
        m=re.match(fa3ilRegex,form)
        if m:
            # print (i,m.group(0),"\t        fa3il, racine :",m.group(1),m.group(2),m.group(3))                    
            formeDict=extractDict(m)
            formeDict["gabarit"]=["fa3il"]
            pdFormeDict=pd.DataFrame([formeDict])
            dfIntensifs=pd.concat([dfIntensifs,pdFormeDict])
dfIntensifs=dfIntensifs.reset_index().drop(columns="index")
dfIntensifs  

,forme,gabarit,c3,c2,c1
0,جديد,"[fa3iil, fa3il]",د,د,ج
1,كبير,"[fa3iil, fa3il]",ر,ب,ك
2,ثلاث,[fa33aal],ث,ل,ث
3,أخير,"[fa3iil, fa3il]",ر,خ,أ
4,عديد,"[fa3iil, fa3il]",د,د,ع
...,...,...,...,...,...
592,رضيض,"[fa3iil, fa3il]",ض,ض,ر
593,مصوح,[fa3uul],ح,ص,م
594,لفوح,[fa3uul],ح,ف,ل
595,نزيح,"[fa3iil, fa3il]",ح,ز,ن


In [24]:

dfIntensifs.to_csv(path_or_buf="ADJ-Extraction-%s.tsv"%source,encoding="utf8",sep="\t")

# Fin

In [25]:
dfIntensifs[dfIntensifs["forme"].str.endswith(marbotat)]

,forme,gabarit,c3,c2,c1
99,شقيقة,"[fa3iil, fa3il]",ق,ق,ش
158,بسيطة,"[fa3iil, fa3il]",ط,س,ب
380,مشاحة,[fa33aal],ح,ش,م
577,شجيعة,"[fa3iil, fa3il]",ع,ج,ش


In [10]:
dfIntensifs.iloc[164,0]==dfIntensifs.iloc[165,0]

False